# Analytical guided modes of a step-index fiber 

The cross section of a step-index fiber, modeled in the $xy$-plane, consists of a circular core surrounded by a circular cladding. The refractive index $n(x, y)$ of such a fiber is a piecewise constant function taking one value in the core and another in the cladding. The purpose of this notebook is to show how guided modes of such a fiber can be computed semi-analytically using `fibermode` facilities. 

## Guided modes 

The guided modes of a waveguide are functions $\varphi_i(x, y)$, each of which, together with their real propagation constant $\beta_i$,  satisfy the two-dimensional  Helmholtz eigenvalue equation in the $xy$-plane, 
$$
	(\Delta + k^2 n^2) \varphi_i = \beta_i^2 \varphi_i,
$$
for some given wavenumber $k$  determined by the operating wavelength. Guided modes decay exponentially out in the cladding. Then solutions of the form $u(x, y, z) = e^{i \beta_m z} \varphi_m(x, y)$ solve the three-dimensional Helmholtz equation of a wave  propagating along the longitudinal ($z$) direction of the fiber.

For step-index fibers, such guided modes $\varphi_i$ are called LP (linearly polarized) modes, and are indexed by two integers $\ell, m$ (instead of the single $i$). Such $\varphi_{\ell m}$ are referred to as $LP_{\ell m}$-modes in the optics literature. The $m$-index ($m\ge 1$) indicates the angular variations and the $\ell$-index ($\ell\ge 0$) specifies the radial variations. Expressions for the $LP_{\ell m}$ modes can be computed exactly in closed form (see e.g., [Reider's book](https://doi.org/10.1007/978-3-319-26076-1))
in terms of the standard Bessel function  $\mathcal{J}_\ell$ and  the modified Bessel function $\mathcal{K}_\ell$ of the second kind.  The closed-form expressions are obtained after solving the   so-called *characteristic equation* of the fiber. 

Setting the fiber's "numerical aperture"  $NA = \sqrt{n_{\text{core}}^2-n_{\text{clad}}^2}$, we obtain the characteristic equation

$$
X\mathcal{J}_{l-1}(X)\mathcal{K}_{l}(\sqrt{NA^2-X^2})+\sqrt{NA^2-X^2}\mathcal{J}_{l}(X)\mathcal{K}_{l}(\sqrt{NA^2-X^2})
$$

and solve this equation using a bisection-based root-finding method. For each $l$, let the roots of the characteristic equation be enumerated as $X_{lm}$, and the propagation constants are given by:

$$
\beta_{lm}=\sqrt{n_{\text{core}}^2k_s^2-X_{lm}^2r^2_{\text{core}}}
$$

Set $\mathcal{R}_{lm}=X_{lm}/r_{\text{core}}$ and $\mathcal{G}_{lm}=\sqrt{\beta_{lm}-n_{\text{clad}}k_s^2}$. The corresponding $LP_{lm}$ modes take the following form in polar coordinates:
    $$
    \varphi_{lm}(r,\theta)=\begin{cases}
    \mathcal{K}_l(\mathcal{G}_{lm}r_{\text{core}})\mathcal{J}_l(\mathcal{R}_{lm}r)\cos(i\theta), & 0 \leq r < r_{\text{core}}\\
    \mathcal{J}_l(\mathcal{R}_{lm}r_{\text{core}})\mathcal{K}_l(\mathcal{G}_{lm}r)\cos(i\theta), &r_{\text{core}}\leq r \leq r_{\text{clad}}\end{cases}
    $$

## Making a step-index fiber object

A number of realistic optical fiber parameters are stored in `fibermode` and accessible by their name. Here is the list of all available names.

In [ ]:
from fibermode import named_stepindex_fibers

list(named_stepindex_fibers.keys())

One way to make a step-index fiber model is by using one of these names, as follows.

In [ ]:
from fibermode import StepIndexExact

f = StepIndexExact('Nufern_Yb')
print(f)

One can now ask the object `f` for its guided modes and query further properties. All methods are seen in `help(StepIndexExact)`. 

Another way to make a step-index fiber object is  by  providing the minimal necessary parameter values (instead of a preset name). This is shown next.

## A single-mode fiber

In [ ]:
fib = StepIndexExact(rcore=1, rclad=10, ncore=1, NA=0.5, ks=4)
print(fib)

Comparing with the previous values, you can see that this fiber does not have realistic geometry. However, in computing the modes, instead of such dimensions, the critical non-dimensional parameter of importance is the $V$-number of the fiber. This value printed above, is not so much different from that of the previous realistic `Nufern_Yb` fiber.

If the $V$-number is below the first Bessel root $\approx 2.4$, by the theory of step-index fibers, we expect the fiber to be a single-mode fiber, i.e.,  we expect to find only one single $LP_{\ell m}$ mode, with $\ell=0$ and $m=1$, and no more modes. The `StepIndexExact` class has a method `propagation_constants`$(\ell)$ to find the $\beta_{\ell m}$ corresponding to the $LP_{\ell m}$ mode.


In [ ]:
help(StepIndexExact.propagation_constants)

In [ ]:
X = fib.propagation_constants(0)
X

If higher values of  $\ell$ are given as input, this method returns nothing (except for various one-time warnings) for this fiber, confirming that this fiber is a single-mode fiber.

In [ ]:
fib.propagation_constants(1)

The `X`-value found above contains the propagation constant in a non-dimensional form. To obtain the value of the corresponding $\beta$, use the function `fib.XtoBeta(X)`. We will do this in the next example of a fiber with many more modes.

## Guided modes of an LMA fiber

A Large Mode Area (LMA) fiber, designed to carry more energy through a wider cross-section, has a high $V$-number, and admits many guided modes. An example of such a fiber is the ytterbium-doped fiber `Nufern_Yb`.

In [ ]:
f = StepIndexExact('Nufern_Yb')

This fiber has two modes for the $\ell=0$ case, 
$$
\beta_{01}, \quad \beta_{02}
$$
whose values are computed as follows.

In [ ]:
X0 = f.propagation_constants(0)
X0

In addition it has more modes with $\ell=1$:

In [ ]:
X1 = f.propagation_constants(1)
X1

It also has modes with $\ell=2$:

In [ ]:
X2 = f.propagation_constants(2)
X2

The search for propagation constants with $\ell=3$ fails, indicating no more modes.

In [ ]:
X3 = f.propagation_constants(3)
X3

The physical propagation constants $\beta_{\ell m}$ are obtained from the above non-dimensionalized $X_{\ell m}$ values  as follows:

**Propagation constants for $\ell=0$:**   $\qquad \beta_{0 m}$ values

In [ ]:
f.XtoBeta(X0)

**Propagation constants for $\ell=1$:**  $\qquad \beta_{1 m}$ values

In [ ]:
f.XtoBeta(X1)

**Propagation constants for $\ell=2$:**  $\qquad \beta_{2 m}$ values

In [ ]:
f.XtoBeta(X2)

These are all the propagation constants of this fiber's guided modes.

## Displaying guided modes

## The root finding behind the computation